In [1]:
import pandas as pd
import numpy as np
import re
import ast
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

""" Extra Imports"""
## I just hate Future Warning so for Ignoring Them
import warnings
warnings.filterwarnings('ignore')

## For make jupyter full screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
## Data base Kaggel
'''
https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata?resource=download&select=tmdb_5000_movies.csv
'''

In [ ]:
## read csv and merge into DF
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
df = movies.merge(credits, on='title')
df

In [ ]:
## Only useful columns are taken from dataFrame
new_df = df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
## check null rows and drop it
new_df.isnull().sum()
new_df.dropna(inplace=True)

# Data Cleaning

In [7]:
'''The above code is converting the genre column from a string to a list.'''
def converter(obj):
    genre_list = []
    for i in ast.literal_eval(obj):
        genre_list.append(i['name'])
    return genre_list

'''The above code is converting the list of dictionaries into a list of strings.'''
new_df['genres'] = new_df['genres'].apply(converter)
new_df['keywords'] = new_df['keywords'].apply(converter)
new_df

In [9]:
'''The above code is taking the first 5 input from the list of input and returning them as a list.'''
def converter_5(obj):
    genre_list = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 5:
            genre_list.append(i['name'])
            counter += 1
        else:
            break
    return genre_list

'''The above code is converting the cast column into a list of strings.'''
new_df['cast'] = new_df['cast'].apply(converter_5)
new_df

In [11]:
'''A function that takes in a string and returns a list of directors.'''
def converterD(obj):
    genre_list = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            genre_list.append(i['name'])
            break
    return genre_list

'''The above code is converting the crew column into a dictionary.'''
new_df['crew'] = new_df['crew'].apply(converterD)
new_df['overview'] = new_df['overview'].apply(lambda x:x.split())
new_df

In [13]:
for j in new_df.loc[:, ~new_df.columns.isin(['movie_id', 'title'])]:
    new_df[j] = new_df[j].apply(lambda x: [i.strip().replace(" ", "") for i in x])
new_df    

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui...","[Action, Crime, Thriller]","[unitedstates–mexicobarrier, legs, arms, paper...","[CarlosGallardo, JaimedeHoyos, PeterMarquardt,...",[RobertRodriguez]
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended...","[Comedy, Romance]",[],"[EdwardBurns, KerryBishé, MarshaDietlein, Cait...",[EdwardBurns]
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,...","[Comedy, Drama, Romance, TVMovie]","[date, loveatfirstsight, narration, investigat...","[EricMabius, KristinBooth, CrystalLowe, GeoffG...",[ScottSmith]
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is...",[],[],"[DanielHenney, ElizaCoupe, BillPaxton, AlanRuc...",[DanielHsia]


In [14]:
new_df['tags'] = new_df['overview'] + new_df['genres'] + new_df['keywords'] + new_df['cast'] +new_df['crew']
tmp_df = new_df[['movie_id','title','tags']]
tmp_df

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [15]:
tmp_df['tags'] = tmp_df['tags'].apply(lambda x:' '.join(x).lower())
tmp_df['tags'] = tmp_df['tags'].str.replace(r"\d+|\W|\_",' ',regex=True)
tmp_df

,movie_id,title,tags
0,19995,Avatar,in the nd century a paraplegic marine is dis...
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believed to be dead ha...
2,206647,Spectre,a cryptic message from bond s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,john carter is a war weary former military ca...
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple s honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered",signed sealed delivered introduces a dedic...
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


# Stemmer

In [16]:
ps = PorterStemmer()

In [17]:
## Remove tenses of all words and make it simpler tense

def stem_text(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [18]:
tmp_df['tags'] = tmp_df['tags'].apply(stem_text)

# Model

In [19]:
cv = CountVectorizer(max_features=3500, stop_words='english')
vectors = cv.fit_transform(tmp_df['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [20]:
cv.get_feature_names_out()

array(['aaron', 'aaroneckhart', 'abandon', ..., 'zone', 'zoo',
       'zooeydeschanel'], dtype=object)

In [ ]:
similarity = cosine_similarity(vectors)
similarity

In [ ]:
## Give Best top 10 matching movie recommendation

def recommend(movie):
    movie_index = tmp_df[tmp_df['title'] == movie].index[0]
    distance = similarity[movie_index]
    movie_lst = sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:11]
    for i in movie_lst:
        print(tmp_df.iloc[i[0]].title)

In [ ]:
## OutPut
recommend('John Carter')